# Figure 2

In [5]:
%run 'stability_fig_init.py'

In [6]:
# 'rgb(0,139,251)'
# 'rgb(255,0,81)'

import colorcet as cc

color = px.colors.qualitative.T10[0]
color2 = px.colors.qualitative.T10[2]
# color = cc.CET_D1A[::16][3]
# color2 = cc.CET_D1A[::16][-3]


In [7]:
cc.CET_D1A[5]


'#192e80'

In [8]:
predictors = ["daily_profile", "gtb_full"]
model_names = ["Daily profile", "GTB (full model)"]

fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=["<b>{}</b>".format(i) for i in area_names],
    shared_yaxes=True,
    horizontal_spacing=0.01,
)
for i, area in enumerate(areas):
    scores = pd.DataFrame(index=targets, columns=model_names, dtype="float")

    for j, targ in enumerate(targets):
        y_train = pd.read_hdf(version_folder.format(area, version) + "y_train.h5").loc[
            :, targ
        ]
        X_train = pd.read_hdf(version_folder.format(area, version) + "X_train_full.h5")
        X_test = pd.read_hdf(version_folder.format(area, version) + "X_test_full.h5")

        daily_profile = y_train.groupby(X_train.index.time).mean()

        y_test = pd.read_hdf(version_folder.format(area, version) + "y_test.h5").loc[
            :, targ
        ]
        y_pred = pd.read_hdf(fit_folder.format(area, version, targ) + "y_pred.h5")
        y_pred["daily_profile"] = [daily_profile[time] for time in X_test.index.time]

        scores.loc[targ, :] = [
            r2_score(y_test, y_pred.loc[:, pred]) for pred in predictors
        ]

    patterns_scores = {
        "CE": [
            0.7366887734219272,
            0.3460977575087052,
            0.46570919676000955,
            0.39744821762198324,
        ],
        "Nordic": [
            0.4624508196230478,
            0.27321751015117923,
            0.15804190856742217,
            0.3932313342404363,
        ],
        "GB": [
            0.22114800956884373,
            0.12038526161152674,
            0.17563018282503184,
            0.22023701831395204,
        ],
    }

    if i == 0:
        fig.add_trace(
            go.Bar(
                x=[target_name[j] for j in scores.index.values],
                y=scores["Daily profile"],
                marker_color="grey",
                name="Daily profile",
                legendgroup="group1",
            ),
            row=1,
            col=i + 1,
        )
        fig.add_trace(
            go.Bar(
                x=[target_name[j] for j in targets],
                y=patterns_scores[area],
                marker_color=color,
                name="w/o HVDC flows",
                legendgroup="group2",
            ),
            row=1,
            col=i + 1,
        )
        fig.add_trace(
            go.Bar(
                x=[target_name[j] for j in scores.index.values],
                y=scores["GTB (full model)"],
                marker_color=color2,
                name="with HVDC flows",
                legendgroup="group3",
                texttemplate="%{text:.1f}",
                text=np.round(scores["GTB (full model)"] / patterns_scores[area], 1),
                textposition="outside",
            ),
            row=1,
            col=i + 1,
        )
    else:
        fig.add_trace(
            go.Bar(
                x=[target_name[j] for j in scores.index.values],
                y=scores["Daily profile"],
                marker_color="grey",
                legendgroup="group1",
                showlegend=False,
            ),
            row=1,
            col=i + 1,
        )
        fig.add_trace(
            go.Bar(
                x=[target_name[j] for j in targets],
                y=patterns_scores[area],
                marker_color=color,
                legendgroup="group2",
                showlegend=False,
            ),
            row=1,
            col=i + 1,
        )
        fig.add_trace(
            go.Bar(
                x=[target_name[j] for j in scores.index.values],
                y=scores["GTB (full model)"],
                marker_color=color2,
                legendgroup="group3",
                showlegend=False,
                texttemplate="%{text:.1f}",
                text=np.round(scores["GTB (full model)"] / patterns_scores[area], 1),
                textposition="outside",
            ),
            row=1,
            col=i + 1,
        )

fig.update_layout(template="plotly_white", width=1240, height=400)
# fig.update_layout(template='plotly_white', width=1650, height=550)
fig.update_layout(
    xaxis=dict(
        linewidth=1,
        linecolor="black",
        mirror=True,
        ticks="outside",
        showline=True,
        gridwidth=0,
        tick0=0,
        dtick=1,
    ),
    yaxis=dict(
        title_text="R<sup>2</sup> Score",
        title_standoff=25,
        linewidth=1,
        linecolor="black",
        mirror=True,
        ticks="outside",
        showline=True,
        gridwidth=1,
        range=[0, 0.85],
        tickmode="array",
        tickvals=[0.2, 0.4, 0.6, 0.8],
    ),
    xaxis2=dict(
        linewidth=1,
        linecolor="black",
        mirror=True,
        ticks="outside",
        showline=True,
        gridwidth=0,
        tick0=0,
        dtick=1,
    ),
    yaxis2=dict(
        linewidth=1,
        linecolor="black",
        mirror=True,
        ticks="",
        showline=True,
        gridwidth=1,
        tickmode="array",
        tickvals=[0.2, 0.4, 0.6, 0.8],
    ),
    xaxis3=dict(
        linewidth=1,
        linecolor="black",
        mirror=True,
        ticks="outside",
        showline=True,
        gridwidth=0,
        tick0=0,
        dtick=1,
    ),
    yaxis3=dict(
        linewidth=1,
        linecolor="black",
        mirror=True,
        ticks="",
        showline=True,
        gridwidth=1,
        tickmode="array",
        tickvals=[0.2, 0.4, 0.6, 0.8],
    ),
)
fig.update_layout(
    legend=dict(
        yanchor="bottom",
        y=-0.3,
        # itemsizing='constant',
        # itemwidth=50
        # xanchor='',
        # x=0.98
    )
)

fig.update_layout(font=dict(size=20, color="black"), legend_orientation="h")
# fig.update_annotations(font_size=37)
# fig.update_layout(uniformtext_minsize=37)
fig.update_layout(margin=dict(b=85))
# fig.update_layout(font=dict(size=20, color='black'))
fig.update_annotations(font_size=20)
fig.update_layout(legend_font_size=20)
# fig.update_layout(legend_y=-0.3)
fig.update_layout(margin_t=30)
fig.update_layout(margin_r=50)

print("creating plot...")
scope = pio.kaleido.scope
scope._shutdown_kaleido()
pio.write_image(fig, "fig2.pdf", format="pdf", validate=False, engine="kaleido")
time.sleep(1)
pio.write_image(fig, "fig2.pdf", format="pdf", validate=False, engine="kaleido")
scope._shutdown_kaleido()


creating plot...
